In [11]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
import datetime

In [12]:
Options = Options()
Options.add_argument('--headless')
driver = webdriver.Chrome(options=Options)

In [13]:
url_base = f"https://a24.com"

In [14]:
soup = []
limite_paginado = 12 #la pagina deja de devolver resultados luego de este limite
for i in range(0,limite_paginado): 
    print(i, end='\r')
    driver.get(f"{url_base}/contenidos/resultado.html?search=milei#gsc.q=milei&gsc.page={i}")
    pag =  driver.page_source
    soup.append(BeautifulSoup(pag, 'html5lib'))

In [15]:
paginas = []
for i in range(0,limite_paginado): 
    paginas.append(soup[i].find_all('div', class_='gsc-webResult gsc-result'))

In [16]:
noticias = pd.DataFrame(columns=['indice','titulo','fecha','link','source','contenido','copete'])

i = -1
for pagina in paginas:
    for noticia in pagina:
        i = i + 1
        noticias.loc[i,'indice'] = i
        #noticias.loc[i,'titulo'] = (noticia.get_text())
        noticias.loc[i,'link'] = f"{noticia.find('a')['href']}"
        print(f'Procesado articulo {i}', end='\r')
        #print(f"Articulo: {i} Titulo: \n{titulo}\nLink:\n{link}\n-------------------------------------------------------------------------\n")


In [17]:
for index, row in noticias.iterrows():
    print(f"Articulo {row.indice} - {row.link}", end='\r')
    driver.get(row.link)
    pag =  driver.page_source
    row.source = BeautifulSoup(pag, 'html5lib')

In [18]:
noticias = noticias[~noticias['link'].str.contains('tags')]

for index, row in noticias.iterrows():
    #print(f"Articulo {row.indice} - {row.link}")
    contenido_source = row.source.find_all('div', class_=lambda x: x and (x.startswith('article-content')))#, class_='article-body')
    row.contenido = ''
    for a in contenido_source:
        row.contenido += a.find('p').get_text() + '\n'
    
    header_source = row.source.find('section', class_='news-body-hero')
    row.titulo = header_source.find('h1',class_ = 'news-hero-title').get_text()
    row.fecha = header_source.find('div',class_ = 'news-hero-author-name').get_text()
    row.copete = header_source.find('div',class_ = 'news-hero-description').get_text()

In [19]:
noticias.drop('source', axis = 1, inplace=True)
noticias.to_excel('01_A24.xlsx')

In [20]:
driver.close()